In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

# from torchtext.datasets import TranslationDataset, Multi30k
# from torchtext.data import Field, BucketIterator

# import spacy

import pickle
import random
import math
import os
import time
import nltk

In [27]:
# dir = 'C:/Users/evan_/OneDrive/Documents/School/Graduate/Year 1/Spring/CS 690D/Project/data/processed/'
dir = 'C:/Users/evan_/Documents/School/Graduate/Year 1/Spring/CS 690D/Project/Base Model/data/processed/'


with open(dir + 'english_no_pad_sorted_50k.pickle', 'rb') as handle:
    english = pickle.load(handle)
    
with open(dir + 'german_no_pad_sorted_50k.pickle', 'rb') as handle:
    german = pickle.load(handle)

In [28]:
max_length_en = len(english['train'][-1])
max_length_de = len(german['train'][-1])

batch_size = max((max_length_en, max_length_de))

# Method 1 for batching, I don't think it is correct

In [69]:
def get_batches(data, b_sz):
    batches = [[]]
    
    for sent in data:
        cur_len = 0
        for b in batches[-1]:
            cur_len += len(b)
            
        # if the sentence + current length is not greater than the batch size, then add it to the batch
        # otherwise fill the current batch
        if (cur_len + len(sent)) <= b_sz: 
            batches[-1].append(sent)
        else:
            dif = b_sz - cur_len
            if dif > 0:
                pad_list = [0] * dif
                batches[-1].append(pad_list)
            batches.append([])        
            batches[-1].append(sent)
    
    # Don't forget about the last batch! make sure it is of the proper length too
    cur_len = 0
    for b in batches[-1]:
        cur_len += len(b)
    if cur_len < b_sz:
        dif = b_sz - cur_len
        pad_list = [0] * dif
        batches[-1].append(pad_list)
        
    return batches

batches = get_batches(english['train'], batch_size)

# Method 2 for batching, I think this method is correct

In [31]:
def get_batches(data, b_sz):
    batches = [[]]
    
    for sent in data:
        cur_len = 0
        for b in batches[-1]:
            cur_len += len(b)
            
        # if the sentence + current length is not greater than the batch size, then add it to the batch
        # otherwise fill the current batch
        if (cur_len + len(sent)) <= b_sz: 
            batches[-1].append(sent)
        else:
            batches.append([])        
            batches[-1].append(sent)
    
    
    for b in batches:
        max_len = len(max(b, key=len))
        
        for sent in b:
            dif = max_len - len(sent)
            if dif > 0:
                pad_list = 0 * dif
                sent.append(pad_list)
        
        
    return batches

batches = get_batches(english['train'], 100)

In [32]:
for i in range(5):
    print(batches[i])

[[2, 11, 3, 0], [2, 15, 3, 0], [2, 11, 3, 0], [2, 11, 3, 0], [2, 11, 3, 0], [2, 533, 11, 3], [2, 2344, 11, 3], [2, 2860, 11, 3], [2, 2937, 308, 3], [2, 2344, 11, 3], [2, 101, 191, 3], [2, 4864, 11, 3], [2, 4912, 308, 3], [2, 3144, 11, 3], [2, 7348, 11, 3], [2, 8069, 11, 3], [2, 3144, 11, 3], [2, 2860, 11, 3], [2, 2860, 11, 3], [2, 2770, 11, 3], [2, 2860, 11, 3], [2, 2344, 11, 3], [2, 170, 191, 3], [2, 286, 308, 3], [2, 10074, 11, 3], [2, 10075, 11, 3]]
[[2, 10074, 11, 3], [2, 10075, 11, 3], [2, 289, 11, 3], [2, 924, 11, 3], [2, 2770, 11, 3], [2, 544, 11, 3], [2, 1247, 11, 3], [2, 2897, 191, 3], [2, 1201, 11, 3], [2, 2438, 11, 3], [2, 2344, 11, 3], [2, 1201, 11, 3], [2, 13543, 11, 3], [2, 491, 11, 3], [2, 289, 191, 3], [2, 15351, 308, 3], [2, 101, 191, 3], [2, 2255, 11, 3], [2, 3237, 11, 3], [2, 2344, 11, 3], [2, 2344, 11, 3], [2, 8664, 11, 3], [2, 155, 2587, 3], [2, 12506, 11, 3], [2, 2344, 11, 3]]
[[2, 14834, 11, 3], [2, 3144, 11, 3], [2, 694, 11, 3], [2, 3144, 11, 3], [2, 18571, 11, 